In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 8.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 4.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_8bit_thresh_4 = AutoModelForCausalLM.from_pretrained('aranulunara/outputs', device_map="auto", load_in_8bit=True)
tokenizer = AutoTokenizer.from_pretrained('aranulunara/outputs')

In [ ]:
from peft import LoraConfig, get_peft_model
lora_config = LoraConfig.from_pretrained('aranulunara/outputs')
model = get_peft_model(model_8bit_thresh_4, lora_config)

In [ ]:
from collections import OrderedDict

def remove_partial_repeated_sentences(text, num_words_to_check=5):
    sentences = text.split('. ')
    filtered_sentences = []
    seen_sentence_starts = set()

    for sentence in sentences:
        current_sentence_start = ' '.join(sentence.split()[:num_words_to_check])

        if current_sentence_start not in seen_sentence_starts:
            filtered_sentences.append(sentence)
            seen_sentence_starts.add(current_sentence_start)

    return '. '.join(filtered_sentences)

In [ ]:
from IPython.display import display, Markdown
device = "cuda:0"
def make_inference(context, question):
  batch = tokenizer(f"### CONTEXT\n{context}\n\n### QUESTION\n{question}\n\n### ANSWER\n", return_tensors='pt').to(device)
  # Move the batch to the device
  #device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
  batch = {k: v.to(device) for k, v in batch.items()}
  with torch.cuda.amp.autocast():
    output_tokens = model.generate(**batch, max_new_tokens=200)
    output_text = tokenizer.decode(output_tokens[0], skip_special_tokens=False)
    start_idx = output_text.find("### ANSWER")
    if start_idx != -1:
        answer_text = output_text[start_idx + len("### ANSWER"):].strip()
        # Remove repeated sentences if they exist
        answer_text = remove_partial_repeated_sentences(answer_text, 3)
        # Further clean up if needed
        end_idx = answer_text.find("###")
        if end_idx != -1:
            answer_text = answer_text[:end_idx].strip()
        display(Markdown(answer_text))


In [ ]:
context = "Current fees are around 260 euro per year, of which 210 euro are for academic tutoring and the remainder for admin  fees and medical insurance in the event of a school-related accident. For more information, please see the following section: 'Doctoral School > Useful Information > Regulations > Decree on Public Fees'."
question = "What's the total cost per year?"

make_inference(context, question)

/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


The total cost per year is around 260 euro per year, of which 210 euro are for academic tutoring and the remainder for admin  fees and medical insurance in the event of a school-related accident. For more information, please see the following section: 'Doctoral School > Useful Information > Regulations > Decree on Public Fees'.

In [ ]:
context = "Current fees are around 260 euro per year, of which 210 euro are for academic tutoring and the remainder for admin  fees and medical insurance in the event of a school-related accident. For more information, please see the following section: 'Doctoral School > Useful Information > Regulations > Decree on Public Fees'."
question = "What are admin fees for?"

make_inference(context, question)

The administration fees for the doctoral school are set by the Decree on Public Fees. For more information, please see the following section: 'Doctoral School > Useful Information > Regulations > Decree on Public Fees'.

In [ ]:
context = "Current fees are around 260 euro per year, of which 210 euro are for academic tutoring and the remainder for admin  fees and medical insurance in the event of a school-related accident. For more information, please see the following section: 'Doctoral School > Useful Information > Regulations > Decree on Public Fees'."
question = "How much for academic tutoring?"

make_inference(context, question)

The fee for academic tutoring is 210 euro per year

In [ ]:
context = "Following information is regarding PhD program of University of Balearic Islands: Current fees are around 260 euro per year, of which 210 euro are for academic tutoring and the remainder for admin  fees and medical insurance in the event of a school-related accident. For more information, please see the following section: 'Doctoral School > Useful Information > Regulations > Decree on Public Fees'."
question = "How much does a PhD cost ?"

make_inference(context, question)

The cost of a PhD varies depending on the university and the country. The average cost of a PhD in the United States is around $60,000 per year

In [ ]:
context = "Following information is regarding PhD program of University of Balearic Islands: Current fees are around 260 euro per year, of which 210 euro are for academic tutoring and the remainder for admin  fees and medical insurance in the event of a school-related accident. For more information, please see the following section: 'Doctoral School > Useful Information > Regulations > Decree on Public Fees'."
question = "How much does a PhD cost at University of Balearic Islands?"

make_inference(context, question)

The cost of a PhD program at University of Balearic Islands is around 260 euro per year, of which 210 euro are for academic tutoring and the remainder for admin  fees and medical insurance in the event of a school-related accident. For more information, please see the following section: 'Doctoral School > Useful Information > Regulations > Decree on Public Fees'.

In [ ]:
context = "Following information is regarding PhD program of University of Balearic Islands: Current fees are around 260 euro per year, of which 210 euro are for academic tutoring and the remainder for admin  fees and medical insurance in the event of a school-related accident. For more information, please see the following section: 'Doctoral School > Useful Information > Regulations > Decree on Public Fees'."
question = "Any fees for accidents at PhD program of University of Balearic Islands?"

make_inference(context, question)

The University of Balearic Islands has a doctoral school, which is a public institution. The doctoral school is a public institution, and therefore, it is not possible to provide information about the fees for accidents at the doctoral school. However, the doctoral school has a doctoral accident insurance, which covers the costs of medical treatment and rehabilitation of students who suffer an accident during their studies. The doctoral accident insurance is a public institution, and therefore, it is not possible to provide information about the fees for accidents at the doctoral accident insurance. However

In [ ]:
context = "In this instance, the originals must be submitted between 1st November and 31st March of the current year. Until original documents are submitted, the CEP is unable to issue any academic certificate for the master's programme, including the final degree certificate. The original and a photocopy must be submitted to the admin services in the Antoni Maria Alcover i Sureda building (Centre for Postgraduate Studies). You must request an appointment to do this step. Where documents are sent by post, a certified copy of each document must be included. The only stamps accepted are those issued by a registered Spanish notary public, a diplomatic service or the issuing body for the document, as long as payment for the certified stamp is accredited."
question = "When do the originals need to be submitted?"

make_inference(context, question)

The originals must be submitted between 1st November and 31st March of the current year. Until original documents are submitted, the CEP is unable to issue any academic certificate for the master's programme, including the final degree certificate. The original and a photocopy must be submitted to the admin services in the Antoni Maria Alcover i Sureda building (Centre for Postgraduate Studies). You must request an appointment to do this step. Where documents are sent by post, a certified copy of each document must be included. The only stamps accepted are those issued by a registered Spanish notary public, a diplomatic service or the issuing body for the document, as long as payment for the certified stamp is accredited.

In [ ]:
context = "On a PhD programme at University of Balearic Islands, there are no classes per se, although depending on the programme where you are registered, there may be some face-to-face activities.There may be two types of activities: Cross-cutting: the academic commissions for each PhD programme decide on whether these activities are obligatory or elective for students. They are organised by the Doctoral School and are common to all PhD programmes. For more information, please see the Doctoral School Training Activities. Specific: the academic commissions for each PhD programme decide on whether these activities are obligatory or elective, and whether they are specific to their students. For more information about these activities, please see the PhD studies on offer section. You will need to select your PhD programme and go to the 'Training Activities' section. If you have any academic questions, you may contact the programme coordinator via the contact form in the 'General Information' section for the programme."
question = "Do I need to take face-to-face classes on a PhD programme at University of Balearic Islands?"

make_inference(context, question)

Yes, you will need to take face-to-face classes on a PhD programme at University of Balearic Islands. You will need to select your PhD programme and go to the 'Training Activities' section. If you have any academic questions, you may contact the programme coordinator via the contact form in the 'General Information' section for the programme.

In [ ]:
context = "At University of Balearic Islands Both the thesis tutor and supervisor are assigned by the academic commission. Your pre-registration application must include a summary of your research project and area. You will also need to make your own thesis supervisor proposal, which will be taken into account when assigning your thesis supervisor(s)."
question = "How are thesis tutors and supervisors assigned at University of Balearic Islands?"

make_inference(context, question)

The thesis tutor and supervisor are assigned by the academic commission. Your pre-registration application must include a summary of your research project and area. You will also need to make your own thesis supervisor proposal, which will be taken into account when assigning your thesis supervisor(s).

In [ ]:
context = "The University of Balearic Islands has established a general academic progress plan on a master's programme, depending on whether students are full- or part-time: Full-time master's students must pass at least 40 percent of their enrolled credits (rounded up) for the academic year, Part-time master's students must pass at least 10 percent of their enrolled credits (rounded up) for the academic year, Where students only enrol for one subject, the aforementioned rules do not apply. More information is available at the following link: 'Information on Academic Regulations, Academic Progress and Attendance Requirements and the Minimum Number of Initial Enrolled Credits'."
question = "What is the minimum number of credits I must pass on a master's programme to continue studying at the University of Balearic Islands?"

make_inference(context, question)

The minimum number of credits required to continue studying at the University of Balearic Islands is 40 credits (rounded up) for full-time students and 10 credits (rounded up) for part-time students. More information is available at the following link: 'Information on Academic Regulations, Academic Progress and Attendance Requirements and the Minimum Number of Initial Enrolled Credits'.

In [ ]:
context = "At the University of Balearic Islands PhD candidates undertake and produce original research in any field of knowledge: PhD thesis. Doctorands must pass annual assessments throughout their enrolment period (initial and subsequent assessments) that serve to monitor their research."
question = "What does a PhD comprise at the University of Balearic Islands"

make_inference(context, question)

The PhD is a research degree awarded by the University of Balearic Islands. It is a four-year degree that is awarded to candidates who have successfully completed a master's degree and have demonstrated a high level of research competence

In [ ]:
context = "At the University of Balearic Islands, official master's degrees: their clear and explicit designation is 'Master's Degree in...'. These programmes aim for students to acquire advanced, specialised or multidisciplinary academic training geared towards an academic or professional specialisation, or to promote the start of research activity. These degrees are official and valid across Spain, with full academic recognition, and enable students to work in regulated professions in accordance with applicable regulations in each instance. Passing an official master's programme grants you the right to obtain the degree certificate with the specific RUCT designation (University and Degree Registry at the Ministry of Education, Culture and Sport). Unofficial master's degrees: these programmes may not use the designation 'Master's Degree in…'. They are included in the postgraduate programme selection, are UIB-specific and represent an additional academic option. These master's programmes and other UIB-specific courses (University Specialist/Expert) enable the university to provide an agile quality response to academic requirements, as well as refresher courses for students (for society in general and for employment opportunities). They are an essential element in our education system and provide top-flight and necessary training."
question = "What is the difference between an official and unofficial master's degree at the University of Balearic Islands?"

make_inference(context, question)

Official master's degrees are those that are officially recognised by the Ministry of Education, Culture and Sport (RUCT) and are valid across Spain. They are designed to provide students with advanced, specialised or multidisciplinary academic training geared towards an academic or professional specialisation, or to promote the start of research activity. Unofficial master's degrees are those that are not officially recognised by the Ministry of Education, Culture and Sport (RUCT) and are not valid across Spain. They are included in the postgraduate programme selection, are UIB-specific and represent an additional academic option. They are an essential element in our education system and provide top-flight and necessary training.

In [ ]:
context = "At the University of Balearic Islands, official master's degrees: their clear and explicit designation is 'Master's Degree in...'. These programmes aim for students to acquire advanced, specialised or multidisciplinary academic training geared towards an academic or professional specialisation, or to promote the start of research activity. These degrees are official and valid across Spain, with full academic recognition, and enable students to work in regulated professions in accordance with applicable regulations in each instance. Passing an official master's programme grants you the right to obtain the degree certificate with the specific RUCT designation (University and Degree Registry at the Ministry of Education, Culture and Sport). Unofficial master's degrees: these programmes may not use the designation 'Master's Degree in…'. They are included in the postgraduate programme selection, are UIB-specific and represent an additional academic option. These master's programmes and other UIB-specific courses (University Specialist/Expert) enable the university to provide an agile quality response to academic requirements, as well as refresher courses for students (for society in general and for employment opportunities). They are an essential element in our education system and provide top-flight and necessary training."
question = "¿Cuál es la diferencia entre un máster oficial y no oficial en la Universidad de las Islas Baleares?"

make_inference(context, question)

The difference between an official and an unofficial master's degree is that the former is a master's degree that is recognised by the RUCT (University and Degree Registry at the Ministry of Education, Culture and Sport). The latter is a master's degree that is not recognised by the RUCT. These master's programmes and other UIB-specific courses (University Specialist/Expert) enable the university to provide an agile quality response to academic requirements, as well as refresher courses for students (for society in general and for employment opportunities). They are an essential element in our education system and provide top-flight and necessary training.

In [ ]:
context = "On a PhD programme at University of Balearic Islands, there are no classes per se, although depending on the programme where you are registered, there may be some face-to-face activities.There may be two types of activities: Cross-cutting: the academic commissions for each PhD programme decide on whether these activities are obligatory or elective for students. They are organised by the Doctoral School and are common to all PhD programmes. For more information, please see the Doctoral School Training Activities. Specific: the academic commissions for each PhD programme decide on whether these activities are obligatory or elective, and whether they are specific to their students. For more information about these activities, please see the PhD studies on offer section. You will need to select your PhD programme and go to the 'Training Activities' section. If you have any academic questions, you may contact the programme coordinator via the contact form in the 'General Information' section for the programme."
question = "¿Dónde se encuentra Universidad de las Islas Baleares?"

make_inference(context, question)

The University of Balearic Islands is located in Palma de Mallorca, Spain. It is a public university with a degree-granting status. It is part of the University of Balearic Islands, which is a public university with a degree-granting status